# Deploying and Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid

import boto3
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Create campaigns <a class="anchor" id="create"></a>
[Back to top](#top)

A campaign is a hosted solution version; an endpoint which you can query for recommendations. Pricing is set by estimating throughput capacity (requests from users for personalization per second). When deploying a campaign, you set a minimum throughput per second (TPS) value. This service, like many within AWS, will automatically scale based on demand, but if latency is critical, you may want to provision ahead for larger demand. For this POC and demo, all minimum throughput thresholds are set to 1. For more information, see the [pricing page](https://aws.amazon.com/personalize/pricing/).

Let's start deploying the campaigns.

### HRNN

Deploy a campaign for your HRNN solution version. It can take around 10 minutes to deploy a campaign. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create multiple campaigns. So we will set up the while loop for all of the campaigns further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [4]:
hrnn_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-hrnn",
    solutionVersionArn = hrnn_solution_version_arn,
    minProvisionedTPS = 1
)

hrnn_campaign_arn = hrnn_create_campaign_response['campaignArn']
print(json.dumps(hrnn_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-southeast-2:002268754244:campaign/personalize-poc-hrnn",
  "ResponseMetadata": {
    "RequestId": "57f5d9dc-f328-47bf-9a10-7025e36fd18b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:55:43 GMT",
      "x-amzn-requestid": "57f5d9dc-f328-47bf-9a10-7025e36fd18b",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS

Deploy a campaign for your SIMS solution version. It can take around 10 minutes to deploy a campaign. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create multiple campaigns. So we will set up the while loop for all of the campaigns further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [5]:
sims_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-SIMS",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = sims_create_campaign_response['campaignArn']
print(json.dumps(sims_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-southeast-2:002268754244:campaign/personalize-poc-SIMS",
  "ResponseMetadata": {
    "RequestId": "98844f08-aebe-4d9e-b037-56de8701d3a6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:55:43 GMT",
      "x-amzn-requestid": "98844f08-aebe-4d9e-b037-56de8701d3a6",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalized Ranking

Deploy a campaign for your personalized ranking solution version. It can take around 10 minutes to deploy a campaign. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create multiple campaigns. So we will set up the while loop for all of the campaigns further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [6]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-southeast-2:002268754244:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "c4714fe1-16d7-45c1-ae97-972b281f2dae",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:55:44 GMT",
      "x-amzn-requestid": "c4714fe1-16d7-45c1-ae97-972b281f2dae",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### View campaign creation status

As promised, how to view the status updates in the console:

* In another browser tab you should already have the AWS Console up from opening this notebook instance. 
* Switch to that tab and search at the top for the service `Personalize`, then go to that service page. 
* Click `View dataset groups`.
* Click the name of your dataset group, most likely something with POC in the name.
* Click `Campaigns`.
* You will now see a list of all of the campaigns you created above, including a column with the status of the campaign. Once it is `Active`, your campaign is ready to be queried.

Or simply run the cell below to keep track of the campaign creation status.

In [7]:
in_progress_campaigns = [
    hrnn_campaign_arn,
    sims_campaign_arn,
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:ap-southeast-2:002268754244:campaign/personalize-poc-hrnn
Build succeeded for arn:aws:personalize:ap-southeast-2:002268754244:campaign/personalize-poc-rerank
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:ap-southeast-2:002268754244:campaign/personalize-poc-SIMS


## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [8]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/artists.dat', delimiter='\t', index_col=0)

# Render some sample data
items_df.head(5)

,name,url,pictureURL
id,,,
1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


By defining the ID column as the index column it is trivial to return an artist by just querying the ID.

In [9]:
item_id_example = 987
artist = items_df.loc[item_id_example]['name']
print(artist)

Earth, Wind & Fire


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [10]:
def get_artist_by_id(artist_id, artist_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return artist_df.loc[int(artist_id)]['name']
    except:
        return "Error obtaining artist"

Now let's test a few simple values to check our error catching.

In [11]:
# A known good id
print(get_artist_by_id(artist_id="987"))
# A bad type of value
print(get_artist_by_id(artist_id="987.9393939"))
# Really bad values
print(get_artist_by_id(artist_id="Steve"))

Earth, Wind & Fire
Error obtaining artist
Error obtaining artist


Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is an artist. 

So, let's sample some data from our dataset to test our SIMS campaign. Grab 5 random artists from our dataframe.

In [12]:
samples = items_df.sample(5)
samples

,name,url,pictureURL
id,,,
5032,Britt Nicole,http://www.last.fm/music/Britt+Nicole,http://userserve-ak.last.fm/serve/252/46033555...
10783,The Coverband,http://www.last.fm/music/The+Coverband,http://userserve-ak.last.fm/serve/252/41449795...
9522,John Popper,http://www.last.fm/music/John+Popper,http://userserve-ak.last.fm/serve/252/77663.jpg
8775,The BellRays,http://www.last.fm/music/The+BellRays,http://userserve-ak.last.fm/serve/252/410926.jpg
9645,Schock,http://www.last.fm/music/Schock,http://userserve-ak.last.fm/serve/252/60699017...


The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Earth, Wind & Fire).

In [13]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(987),
)

In [14]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_artist_by_id(artist_id=item['itemId']))

The Byrds
Johnny Cash
Neil Young
Lacrimas Profundere
Jethro Tull
Bob Dylan
Amorphis
George Harrison
Motörhead
Bruce Springsteen
John Lennon
The Who
The Rolling Stones


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for our sample collection of artists render in a nice dataframe. Again, let's create a helper function to achieve this.

In [15]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, artist_ID):
    # Get the artist name
    artist_name = get_artist_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(artist_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_artist_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [artist_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with the sample of artists we chose before.

In [16]:
sims_recommendations_df = pd.DataFrame()
artists = samples.index.tolist()

for artist in artists:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, artist)

sims_recommendations_df

,Britt Nicole,The Coverband,John Popper,The BellRays,Schock
0,Britney Spears,Britney Spears,Britney Spears,Britney Spears,Britney Spears
1,Depeche Mode,Depeche Mode,Depeche Mode,Depeche Mode,Depeche Mode
2,Lady Gaga,Lady Gaga,Lady Gaga,Lady Gaga,Lady Gaga
3,Madonna,Madonna,Madonna,Madonna,Madonna
4,Christina Aguilera,Christina Aguilera,Christina Aguilera,Christina Aguilera,Christina Aguilera
5,Muse,Muse,Muse,Muse,Muse
6,The Beatles,The Beatles,The Beatles,The Beatles,The Beatles
7,Rihanna,Rihanna,Rihanna,Rihanna,Rihanna
8,Radiohead,Radiohead,Radiohead,Radiohead,Radiohead
9,Coldplay,Coldplay,Coldplay,Coldplay,Coldplay


You may notice that a lot of the items look the same, hopefully not all of them do. This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### HRNN

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users.

In [17]:
users_df = pd.read_csv(data_dir + '/user_artists.dat', delimiter='\t', index_col=0)
# Render some sample data
users_df.head(5)

,artistID,weight
userID,,
2,51,13883
2,52,11690
2,53,11351
2,54,10300
2,55,8983


In [18]:
users = users_df.sample(3).index.tolist()
users

[695, 391, 436]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [19]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the artist name
    #artist_name = get_artist_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = hrnn_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_artist_by_id(item['itemId'])
        recommendation_list.append(artist)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [20]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample(3).index.tolist()
print(users)

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

[1959, 1078, 8]


,1959,1078,8
0,Britney Spears,Pavement,J.K.
1,Christina Aguilera,Depeche Mode,Depeche Mode
2,Lady Gaga,Whitney Houston,Rihanna
3,Agnetha Fältskog,J.K.,Andrew Sixty
4,Taylor Swift,Rihanna,Lady Gaga
5,Rihanna,Skunk Anansie,Leilah Moreno
6,Madonna,Placebo,Skunk Anansie
7,Eminem,Coldplay,Agnetha Fältskog
8,Pleq,The Killers,Whitney Houston
9,The Pussycat Dolls,Green Day,Paramore


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [21]:
response = personalize.create_event_tracker(
    name='ArtistTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:ap-southeast-2:002268754244:event-tracker/32b5559d
b33c9c67-6393-4db9-bfe3-cb58e301db7b


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [22]:
session_dict = {}

def send_artist_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id):
    # Get the artist name (header of column)
    artist_name = get_artist_by_id(item_id)
    # Interact with the artist
    send_artist_click(USER_ID=user_id, ITEM_ID=item_id)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = hrnn_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_artist_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [artist_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [23]:
# First pick a user
user_id = users_df.sample(1).index.tolist()[0]

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = hrnn_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_artist_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,1858
0,Symphony X
1,Dead Can Dance
2,Stevie Ray Vaughan
3,Neil Young
4,Thievery Corporation
5,Arch Enemy
6,of Montreal
7,Emilie Autumn
8,AFI
9,M83


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [24]:
# Next generate 3 random artists
artists = items_df.sample(3).index.tolist()

In [25]:
# Note this will take about 15 seconds to complete due to the sleeps
for artist in artists:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, artist)
    time.sleep(5)
user_recommendations_df

,1858,Maite Perroni,Dirty Pretty Things,Babyface
0,Symphony X,Symphony X,Symphony X,Symphony X
1,Dead Can Dance,Dead Can Dance,Dead Can Dance,Dead Can Dance
2,Stevie Ray Vaughan,Stevie Ray Vaughan,Stevie Ray Vaughan,Stevie Ray Vaughan
3,Neil Young,Neil Young,Neil Young,Neil Young
4,Thievery Corporation,Thievery Corporation,Thievery Corporation,Thievery Corporation
5,Arch Enemy,Arch Enemy,Arch Enemy,Arch Enemy
6,of Montreal,of Montreal,of Montreal,of Montreal
7,Emilie Autumn,Emilie Autumn,Emilie Autumn,Emilie Autumn
8,AFI,AFI,AFI,AFI
9,M83,M83,M83,M83


In the cell above, the first column after the index is the user's default recommendations from HRNN, and each column after that has a header of the artist that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset. If you wanted to better understand this, try simulating clicking random artists of random genres, and you should see a more pronounced impact.

### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. To demonstrate this, we will need a random user and a random collection of 25 items.

In [26]:
rerank_user = users_df.sample(1).index.tolist()[0]
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [27]:
rerank_list = []
for item in rerank_items:
    artist = get_artist_by_id(item)
    rerank_list.append(artist)
rerank_df = pd.DataFrame(rerank_list, columns = [rerank_user])
rerank_df

,107
0,FLOPPY
1,Spaghetti Vabune!
2,Freddie McGregor
3,David Vade
4,Harland
5,Maxwell
6,Jorge e Mateus Ao Vivo Goiania
7,Basic Element
8,仲野順也
9,Do Amor


Then make the personalized ranking API call.

In [28]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

{'ResponseMetadata': {'RequestId': '33939935-e096-4c38-b4b0-2e96e7acc961',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Thu, 09 Apr 2020 05:05:03 GMT',
   'x-amzn-requestid': '33939935-e096-4c38-b4b0-2e96e7acc961',
   'content-length': '1396',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'personalizedRanking': [{'itemId': '1783'},
  {'itemId': '3914'},
  {'itemId': '15145'},
  {'itemId': '9137'},
  {'itemId': '5549'},
  {'itemId': '18547'},
  {'itemId': '13549'},
  {'itemId': '8670'},
  {'itemId': '18088'},
  {'itemId': '5422'},
  {'itemId': '247'},
  {'itemId': '17367'},
  {'itemId': '6958'},
  {'itemId': '10968'},
  {'itemId': '16107'},
  {'itemId': '13159'},
  {'itemId': '18237'},
  {'itemId': '11306'},
  {'itemId': '17452'},
  {'itemId': '18272'},
  {'itemId': '16443'},
  {'itemId': '10420'},
  {'itemId': '4647'},
  {'itemId': '2296'},
  {'itemId': '284'}]}

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [29]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    artist = get_artist_by_id(item['itemId'])
    ranked_list.append(artist)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,107,Re-Ranked
0,FLOPPY,Esbjörn Svensson Trio
1,Spaghetti Vabune!,Basic Element
2,Freddie McGregor,Ruth Lorenzo
3,David Vade,Odd Nordstoga
4,Harland,Yo La Tengo
5,Maxwell,The Crucified
6,Jorge e Mateus Ao Vivo Goiania,Anthony Callea
7,Basic Element,譚盾
8,仲野順也,Hannah Georgas
9,Do Amor,Spaghetti Vabune!


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example, or if you are filtering on a category and want to show the most likely good items.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [30]:
# Get the user list
batch_users = users_df.sample(3).index.tolist()

# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in batch_users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [31]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "2079"}
{"userId": "482"}
{"userId": "333"}


Upload the file to S3 and save the path as a variable for later.

In [32]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://002268754244personalizepoc/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [33]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://002268754244personalizepoc/


Now just make the call to kick off the batch export process.

In [34]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = hrnn_solution_version_arn,
    jobName = "POC-Batch-Inference-Job-HRNN",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateBatchInferenceJob operation: Another resource with Arn arn:aws:personalize:ap-southeast-2:002268754244:batch-inference-job/POC-Batch-Inference-Job-HRNN already exists.

Run the while loop below to track the status of the batch recommendation call. This can take around 25 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Once the batch recommendations job has finished processing, we can grab the output uploaded to S3 and parse it.

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            artist = get_artist_by_id(item)
            recommendation_list.append(artist)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `04_Clean_Up_Resources.ipynb`.